# RPA trazer informações do banco do 2o para o banco do 1o

In [ ]:
import os
import psycopg2
from pymongo import MongoClient
import pandas as pd
from dotenv import load_dotenv
import pymongo

In [2]:
pip install dotenv 

Note: you may need to restart the kernel to use updated packages.
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'error'


  error: subprocess-exited-with-error
  
  × pip subprocess to install backend dependencies did not run successfully.
  │ exit code: 1
  ╰─> [39 lines of output]
           ---------------------------------------- 0.0/145.4 kB ? eta -:--:--
           -- ------------------------------------- 10.2/145.4 kB ? eta -:--:--
           ------- ----------------------------- 30.7/145.4 kB 325.1 kB/s eta 0:00:01
           ---------- -------------------------- 41.0/145.4 kB 326.8 kB/s eta 0:00:01
           ------------------------- ---------- 102.4/145.4 kB 535.8 kB/s eta 0:00:01
           -----------------------------------  143.4/145.4 kB 655.8 kB/s eta 0:00:01
           ------------------------------------ 145.4/145.4 kB 540.6 kB/s eta 0:00:00
        Installing build dependencies: started
        Installing build dependencies: finished with status 'done'
        Getting requirements to build wheel: started
        Getting requirements to build wheel: finished with status 'done'
        P

In [ ]:
load_dotenv()

host_1ano = os.getenv("DB_HOST_1ano")
port_1ano = int(os.getenv("DB_PORT_1ano"))
user_1ano = os.getenv("DB_USER_1ano")
password_1ano = os.getenv("DB_PASSWORD_1ano")
database_1ano = os.getenv("DB_NAME_1ano")
db_url_1ano = os.getenv("DATABASE_URL_1ano")

host_2ano = os.getenv("DB_HOST_2ano")
port_2ano = int(os.getenv("DB_PORT_2ano"))
user_2ano = os.getenv("DB_USER_2ano")
password_2ano = os.getenv("DB_PASSWORD_2ano")
database_2ano = os.getenv("DB_NAME_2ano")
db_url_2ano = os.getenv("DATABASE_URL_2ano")


conexao = MongoClient(os.getenv("DB_MONGO_URL"))
db_name = conexao[os.getenv("DB_MONGO_NAME")]
collection_funcoes = db_name[os.getenv("DB_MONGO_COLLECTION_FUNCOES")] 
collection_situacao_trabalhista = db_name[os.getenv("DB_MONGO_COLLECTION_SITUACAO_TRABALHISTA")]

In [ ]:
try:
    # conectando-se ao banco do 1ano
    conn_bd1 = psycopg2.connect(
        user=user_1ano,
        host=host_1ano,
        port=port_1ano,
        password=password_1ano,
        database=database_1ano,
    )

    # conectando-se ao banco do 2ano
    conn_bd2 = psycopg2.connect(
        user=user_2ano,
        host=host_2ano,
        port=port_2ano,
        password=password_2ano,
        database=database_2ano,
    )
    
except Exception as e:
    print("Error connecting to PostgreSQL:", e)

In [ ]:
# Criando um cursores para fazer queries em ambos os bancos
cursor_bd1 = conn_bd1.cursor()
cursor_bd2 = conn_bd2.cursor()

## tipo_vaga

In [ ]:
# Lendo os valores da tabela tipo_vaga do 2ano
df_tipo_vaga_bd2 = pd.read_sql_query("SELECT * FROM tipo_vaga;", conn_bd2)

try:
    for i in range(len(df_tipo_vaga_bd2)):
        # Lendo os valores de cada linha do dataframe que contém as informações da tabela tipo_vaga do 2ano
        values2o = (df_tipo_vaga_bd2['id'][i], df_tipo_vaga_bd2['nome'][i])
        
        # Query para atualizar a tabela do 1o com as informações do 2o
        update_query = "UPDATE public.tipo_vaga SET nome = %s WHERE id = %s;"

        # Executando a query de atualização
        cursor_bd1.execute(update_query, values2o)

        # Se a query de atualização não tiver nenhuma linha alterada, então significa que os valores na tabela do 2o são novos
        if cursor_bd1.rowcount == 0:
            # query para inserir
            insert_query = "INSERT INTO public.tipo_vaga(id, nome) VALUES (%s, %s);"

            # Executando a query de insert
            cursor_bd1.execute(insert_query, values2o)

    conn_bd1.commit()  # Commit das alterações no bd1
    conn_bd2.commit()  # Commit das alterações no bd2

except Exception as e:
    print(f"Ocorreu um erro: {e}")

finally:
    # Fechar as conexões com os bancos de dados
    conn_bd2.close()
    conn_bd1.close()


## permissao_vaga

In [ ]:
# Lendo os valores da tabela tipo_vaga do 2ano
df_tipo_vaga_bd2 = pd.read_sql_query("SELECT * FROM permissao_vaga;", conn_bd2)

try:
    for i in range(len(df_tipo_vaga_bd2)):
        # Lendo os valores de cada linha do dataframe que contém as informações da tabela tipo_vaga do 2ano
        values2o = (df_tipo_vaga_bd2['id'][i], df_tipo_vaga_bd2['permissao'][i])
        
        # Query para atualizar a tabela do 1o com as informações do 2o
        update_query = "UPDATE public.permissao_vaga SET permissao = %s WHERE id = %s;"

        # Executando a query de atualização
        cursor_bd1.execute(update_query, values2o)

        # Se a query de atualização não tiver nenhuma linha alterada, então significa que os valores na tabela do 2o são novos
        if cursor_bd1.rowcount == 0:
            # query para inserir
            insert_query = "INSERT INTO public.permissao_vaga(id, permissao) VALUES (%s, %s);"

            # Executando a query de insert
            cursor_bd1.execute(insert_query, values2o)

    # Queries para pegar os IDs da 
    cursor_bd2.execute("SELECT id FROM permissao_vaga;")
    ids_bd2 = [id_[0] for id_ in cursor_bd2.fetchall()]

    # Deletando todos os registros na tabela do 1o que não estão presentes na tabela do 2o
    delete_query = f"""
        DELETE FROM public.permissao_vaga
        WHERE id NOT IN {tuple(ids_bd2)}
    """

    cursor_bd1.execute(delete_query)

    conn_bd1.commit()  # Commit das alterações no bd1
    conn_bd2.commit()  # Commit das alterações no bd2

except Exception as e:
    print(f"Ocorreu um erro: {e}")

finally:
    # Fechar as conexões com os bancos de dados
    conn_bd2.close()
    conn_bd1.close()

## situacao_trabalhista

In [ ]:
# Lendo os valores da tabela tipo_vaga do 2ano
df_tipo_vaga_bd2 = pd.read_sql_query("SELECT * FROM situacao_trabalhista;", conn_bd2)

try:
    for i in range(len(df_tipo_vaga_bd2)):
        # Lendo os valores de cada linha do dataframe que contém as informações da tabela tipo_vaga do 2ano
        values2o = (df_tipo_vaga_bd2['id'][i], df_tipo_vaga_bd2['nome'][i])
        
        # Query para atualizar a tabela do 1o com as informações do 2o
        update_query = "UPDATE public.situacao_trabalhista SET nome = %s WHERE id = %s;"

        # Executando a query de atualização
        cursor_bd1.execute(update_query, values2o)

        # Se a query de atualização não tiver nenhuma linha alterada, então significa que os valores na tabela do 2o são novos
        if cursor_bd1.rowcount == 0:
            # query para inserir
            insert_query = "INSERT INTO public.situacao_trabalhista(id, nome) VALUES (%s, %s);"

            # Executando a query de insert
            cursor_bd1.execute(insert_query, values2o)

    conn_bd1.commit()  # Commit das alterações no bd1
    conn_bd2.commit()  # Commit das alterações no bd2

except Exception as e:
    print(f"Ocorreu um erro: {e}")

finally:
    # Fechar as conexões com os bancos de dados
    conn_bd2.close()
    conn_bd1.close()

## setor

In [ ]:
# Lendo os valores da tabela tipo_vaga do 2ano
df_tipo_vaga_bd2 = pd.read_sql_query("SELECT * FROM setor;", conn_bd2)

try:
    for i in range(len(df_tipo_vaga_bd2)):
        # Lendo os valores de cada linha do dataframe que contém as informações da tabela tipo_vaga do 2ano
        values2o = (df_tipo_vaga_bd2['id'][i], df_tipo_vaga_bd2['nome'][i])
        
        # Query para atualizar a tabela do 1o com as informações do 2o
        update_query = "UPDATE public.setor SET nome = %s WHERE id = %s;"

        # Executando a query de atualização
        cursor_bd1.execute(update_query, values2o)

        # Se a query de atualização não tiver nenhuma linha alterada, então significa que os valores na tabela do 2o são novos
        if cursor_bd1.rowcount == 0:
            # query para inserir
            insert_query = "INSERT INTO public.setor(id, nome) VALUES (%s, %s);"

            # Executando a query de insert
            cursor_bd1.execute(insert_query, values2o)

    conn_bd1.commit()  # Commit das alterações no bd1
    conn_bd2.commit()  # Commit das alterações no bd2

except Exception as e:
    print(f"Ocorreu um erro: {e}")

finally:
    # Fechar as conexões com os bancos de dados
    conn_bd2.close()
    conn_bd1.close()

## tipo_arquivo

In [ ]:
# Lendo os valores da tabela tipo_vaga do 2ano
df_tipo_vaga_bd2 = pd.read_sql_query("SELECT * FROM tipo_arquivo;", conn_bd2)

try:
    for i in range(len(df_tipo_vaga_bd2)):
        # Lendo os valores de cada linha do dataframe que contém as informações da tabela tipo_vaga do 2ano
        values2o = (df_tipo_vaga_bd2['id'][i], df_tipo_vaga_bd2['nome'][i])
        
        # Query para atualizar a tabela do 1o com as informações do 2o
        update_query = "UPDATE public.tipo_arquivo SET nome = %s WHERE id = %s;"

        # Executando a query de atualização
        cursor_bd1.execute(update_query, values2o)

        # Se a query de atualização não tiver nenhuma linha alterada, então significa que os valores na tabela do 2o são novos
        if cursor_bd1.rowcount == 0:
            # query para inserir
            insert_query = "INSERT INTO public.tipo_arquivo(id, nome) VALUES (%s, %s);"

            # Executando a query de insert
            cursor_bd1.execute(insert_query, values2o)
            
    conn_bd1.commit()  # Commit das alterações no bd1
    conn_bd2.commit()  # Commit das alterações no bd2

except Exception as e:
    print(f"Ocorreu um erro: {e}")

finally:
    # Fechar as conexões com os bancos de dados
    conn_bd2.close()
    conn_bd1.close()

## permissao_curso

In [ ]:
# Lendo os valores da tabela tipo_vaga do 2ano
df_tipo_vaga_bd2 = pd.read_sql_query("SELECT * FROM permissao_curso;", conn_bd2)

try:
    for i in range(len(df_tipo_vaga_bd2)):
        # Lendo os valores de cada linha do dataframe que contém as informações da tabela tipo_vaga do 2ano
        values2o = (df_tipo_vaga_bd2['id'][i], df_tipo_vaga_bd2['permissao'][i])
        
        # Query para atualizar a tabela do 1o com as informações do 2o
        update_query = "UPDATE public.permissao_curso SET permissao = %s WHERE id = %s;"

        # Executando a query de atualização
        cursor_bd1.execute(update_query, values2o)

        # Se a query de atualização não tiver nenhuma linha alterada, então significa que os valores na tabela do 2o são novos
        if cursor_bd1.rowcount == 0:
            # query para inserir
            insert_query = "INSERT INTO public.permissao_curso(id, permissao) VALUES (%s, %s);"

            # Executando a query de insert
            cursor_bd1.execute(insert_query, values2o)

    # Queries para pegar os IDs da 
    cursor_bd2.execute("SELECT id FROM permissao_vaga;")
    ids_bd2 = [id_[0] for id_ in cursor_bd2.fetchall()]

    # Deletando todos os registros na tabela do 1o que não estão presentes na tabela do 2o
    delete_query = f"""
        DELETE FROM public.permissao_vaga
        WHERE id NOT IN {tuple(ids_bd2)}
    """

    cursor_bd1.execute(delete_query)

    conn_bd1.commit()  # Commit das alterações no bd1
    conn_bd2.commit()  # Commit das alterações no bd2

except Exception as e:
    print(f"Ocorreu um erro: {e}")

finally:
    # Fechar as conexões com os bancos de dados
    conn_bd2.close()
    conn_bd1.close()

## funcao

In [ ]:
df_funcao_bd2 = pd.DataFrame(list(collection_funcoes.find({})))

try:
    for i in range(len(df_funcao_bd2)):
        # Lendo os valores de cada linha do dataframe que contém as informações da tabela tipo_vaga do 2ano
        values2o = (df_tipo_vaga_bd2['id'][i], df_tipo_vaga_bd2['funcao'][i])
        
        # Query para atualizar a tabela do 1o com as informações do 2o
        update_query = "UPDATE public.permissao_curso SET funcao = %s WHERE id = %s;"

        # Executando a query de atualização
        cursor_bd1.execute(update_query, values2o)

        # Se a query de atualização não tiver nenhuma linha alterada, então significa que os valores na tabela do 2o são novos
        if cursor_bd1.rowcount == 0:
            # query para inserir
            insert_query = "INSERT INTO public.permissao_curso(id, funcao) VALUES (%s, %s);"

            # Executando a query de insert
            cursor_bd1.execute(insert_query, values2o)

    conn_bd1.commit()  # Commit das alterações no bd1
    conn_bd2.commit()  # Commit das alterações no bd2

except Exception as e:
    print(f"Ocorreu um erro: {e}")

finally:
    # Fechar as conexões com os bancos de dados
    conn_bd2.close()
    conn_bd1.close()